In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2013.xlsx', '나주_2014.xlsx', '나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['용봉_2013.xlsx', '용봉_2014.xlsx', '용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2013.xlsx', '우치_2014.xlsx', '우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2013.xlsx
data/yeong/자동/나주_2014.xlsx
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-11-6 23:00
data/yeong/자동/용봉_2013.xlsx
data/yeong/자동/용봉_2014.xlsx
data/yeong/자동/용봉_2015.xlsx
data/yeong/자동/용봉_2016.xlsx
data/yeong/자동/용봉_2017.xlsx
data/yeong/자동/용봉_2018.xlsx
data/yeong/자동/용봉_2019.xlsx
data/yeong/자동/용봉_2020.xlsx


1/1 [==============================] - 0s 20ms/step - gen_loss: 14.6583 - disc_loss: 0.3202 - rmse: 0.4322 - val_loss: 0.4477
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1817 - disc_loss: 0.3566 - rmse: 0.4297 - val_loss: 0.4449
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.4054 - disc_loss: 0.3427 - rmse: 0.4008 - val_loss: 0.4191
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.3511 - disc_loss: 0.3403 - rmse: 0.4172 - val_loss: 0.4617
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.2479 - disc_loss: 0.3185 - rmse: 0.4446 - val_loss: 0.3930
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5054 - disc_loss: 0.3223 - rmse: 0.4652 - val_loss: 0.4199
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.4671 - disc_loss: 0.3480 - rmse: 0.4809 - val_loss: 0.4139
Epoch 56/2000
1/1 [===============

Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1368 - disc_loss: 0.2802 - rmse: 0.3720 - val_loss: 0.4103
Epoch 109/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.1067 - disc_loss: 0.2706 - rmse: 0.3478 - val_loss: 0.3845
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7699 - disc_loss: 0.2870 - rmse: 0.3642 - val_loss: 0.4073
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2362 - disc_loss: 0.2897 - rmse: 0.3673 - val_loss: 0.3922
Epoch 112/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.2871 - disc_loss: 0.2632 - rmse: 0.4240 - val_loss: 0.3738
Epoch 113/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.5729 - disc_loss: 0.2515 - rmse: 0.3583 - val_loss: 0.4356
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0807 - disc_loss: 0.2746 - rmse: 0.3624 - val_loss: 0.4192
Epoch 115/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3995 - disc_loss: 0.2453 - rmse: 0.3982 - val_loss: 0.3692
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9229 - disc_loss: 0.2558 - rmse: 0.3458 - val_loss: 0.3409
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5778 - disc_loss: 0.2345 - rmse: 0.3488 - val_loss: 0.3089
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9246 - disc_loss: 0.2512 - rmse: 0.3301 - val_loss: 0.3381
Epoch 170/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 5.6040 - disc_loss: 0.2283 - rmse: 0.3003 - val_loss: 0.3623
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2063 - disc_loss: 0.2483 - rmse: 0.3299 - val_loss: 0.3408
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1498 - disc_loss: 0.2427 - rmse: 0.3720 - val_loss: 0.3554
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6096 - disc_loss: 0.2362 - rmse: 0.3211 - val_loss: 0.3283
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9933 - disc_loss: 0.2350 - rmse: 0.2631 - val_loss: 0.3352
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5015 - disc_loss: 0.2277 - rmse: 0.3077 - val_loss: 0.3370
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0327 - disc_loss: 0.2049 - rmse: 0.2925 - val_loss: 0.3492
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0546 - disc_loss: 0.2132 - rmse: 0.2952 - val_loss: 0.3056
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9915 - disc_loss: 0.2371 - rmse: 0.3342 - val_loss: 0.3649
Epoch 230/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.4834 - disc_loss: 0.2242 - rmse: 0.3871 - val_loss: 0.3115
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2172 - disc_loss: 0.1516 - rmse: 0.3129 - val_loss: 0.2569
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7771 - disc_loss: 0.1627 - rmse: 0.2960 - val_loss: 0.2762
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2828 - disc_loss: 0.1494 - rmse: 0.2774 - val_loss: 0.3412
Epoch 1735/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8398 - disc_loss: 0.1591 - rmse: 0.2908 - val_loss: 0.2742
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6211 - disc_loss: 0.1680 - rmse: 0.2548 - val_loss: 0.2583
Epoch 1737/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6957 - disc_loss: 0.1460 - rmse: 0.2866 - val_loss: 0.2872
Epoch 1738/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5630 - disc_loss: 0.1579 - rmse: 0.2505 - val_loss: 0.3062
Epoch 1739/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5611 - disc_loss: 0.1547 - rmse: 0.2984 - val_loss: 0.2728
Epoch 1791/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5557 - disc_loss: 0.1557 - rmse: 0.2750 - val_loss: 0.3130
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0308 - disc_loss: 0.1625 - rmse: 0.2537 - val_loss: 0.2681
Epoch 1793/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1911 - disc_loss: 0.1549 - rmse: 0.2388 - val_loss: 0.2743
Epoch 1794/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1026 - disc_loss: 0.1586 - rmse: 0.2625 - val_loss: 0.2634
Epoch 1795/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4423 - disc_loss: 0.1497 - rmse: 0.2980 - val_loss: 0.2739
Epoch 1796/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.8360 - disc_loss: 0.1548 - rmse: 0.2639 - val_loss: 0.2799
Epoch 1797/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1507 - disc_loss: 0.1606 - rmse: 0.2693 - val_loss: 0.2504
Epoch 1849/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2343 - disc_loss: 0.1567 - rmse: 0.2511 - val_loss: 0.2965
Epoch 1850/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3846 - disc_loss: 0.1552 - rmse: 0.2564 - val_loss: 0.2813
Epoch 1851/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.1551 - disc_loss: 0.1569 - rmse: 0.2888 - val_loss: 0.2960
Epoch 1852/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2884 - disc_loss: 0.1468 - rmse: 0.2448 - val_loss: 0.3102
Epoch 1853/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1351 - disc_loss: 0.1461 - rmse: 0.2641 - val_loss: 0.2393
Epoch 1854/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1699 - disc_loss: 0.1467 - rmse: 0.2508 - val_loss: 0.2619
Epoch 1855/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0101 - disc_loss: 0.1561 - rmse: 0.2665 - val_loss: 0.2509
Epoch 1907/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5686 - disc_loss: 0.1529 - rmse: 0.2961 - val_loss: 0.2441
Epoch 1908/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3256 - disc_loss: 0.1590 - rmse: 0.2317 - val_loss: 0.2807
Epoch 1909/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3074 - disc_loss: 0.1616 - rmse: 0.2477 - val_loss: 0.2559
Epoch 1910/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5290 - disc_loss: 0.1492 - rmse: 0.2680 - val_loss: 0.2824
Epoch 1911/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5107 - disc_loss: 0.1564 - rmse: 0.2731 - val_loss: 0.3014
Epoch 1912/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6027 - disc_loss: 0.1617 - rmse: 0.3141 - val_loss: 0.2338
Epoch 1913/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9013 - disc_loss: 0.1582 - rmse: 0.2302 - val_loss: 0.3175
Epoch 1965/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2894 - disc_loss: 0.1596 - rmse: 0.3022 - val_loss: 0.2873
Epoch 1966/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4300 - disc_loss: 0.1603 - rmse: 0.2696 - val_loss: 0.2348
Epoch 1967/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7072 - disc_loss: 0.1515 - rmse: 0.2908 - val_loss: 0.2776
Epoch 1968/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5407 - disc_loss: 0.1517 - rmse: 0.2694 - val_loss: 0.3126
Epoch 1969/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9457 - disc_loss: 0.1588 - rmse: 0.2227 - val_loss: 0.2993
Epoch 1970/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5078 - disc_loss: 0.1581 - rmse: 0.2850 - val_loss: 0.3113
Epoch 1971/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [54]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(68808, 36)
tn


In [55]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (48165, 36) val_df.shape :  (13762, 36) test_df.shape: (6881, 36)


In [56]:
#fake_df.shape[0]/8760

In [57]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [58]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [59]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [60]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong_auto_13/nitrogen/


In [63]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [64]:
rnn_epochs, rnn_in_setps

(20, 240)

## 모델 학습

In [65]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/15
10/10 [==============================] - ETA: 0s - loss: 0.3587 - mean_absolute_error: 0.4529 - nse: 0.2948
Epoch 00001: val_nse improved from -inf to 0.66968, saving model to save/best_model.h5
10/10 [==============================] - 12s 1s/step - loss: 0.3587 - mean_absolute_error: 0.4529 - nse: 0.2948 - val_loss: 0.1640 - val_mean_absolute_error: 0.3126 - val_nse: 0.6697
Epoch 2/15
10/10 [==============================] - ETA: 0s - loss: 0.2139 - mean_absolute_error: 0.3253 - nse: 0.6318
Epoch 00002: val_nse improved from 0.66968 to 0.69353, saving model to save/best_model.h5
10/10 [==============================] - 12s 1s/step - loss: 0.2139 - mean_absolute_error: 0.3253 - nse: 0.6318 - val_loss: 0.1510 - val_mean_absolute_error: 0.3020 - val_nse: 0.6935
Epoch 3/15
10/10 [==============================] - ETA: 0s - loss: 0.1435 - mean_absolute_error: 0.2820 - nse: 0.7084
Epoch 00003: val_nse improved from 0.69353 to 0.72473, saving model to save/best_model.h5
10/10 [===

## core / window.py / 

In [66]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print('nse_val : ', val_nse['GRU'])
#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  3.8231223
predict_day :  5
------------------------
0.4230327 0.33107075 0.5753875
save/yeong/models/yeong_auto_13/nitrogen/
year : 2013 ~ 2020
run :  range(0, 1)
target :  tn   5
length :  68808
train=7, test=1, val=2
nse_val :  [0.70463365]


In [67]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('nse_all : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  4.180541
predict_day :  5
------------------------
0.35830846 0.27214113 0.5216715
nse_all :  [0.7951715]


In [68]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_train : ', val_nse['GRU'])

/home/jaewon/work/venv/water-quality-venv/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  4.142463
predict_day :  5
------------------------
0.3176666 0.22520566 0.47455838
nse_train :  [0.8153397]


In [69]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_test : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  5.224972
predict_day :  5
------------------------
0.52791905 0.47635326 0.6901835
nse_test :  [0.7230159]


In [32]:
#train_df.shape, val_df.shape, test_df.shape

In [33]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
